In [3]:
import numpy as np
import pandas as pd

In [63]:
# Install a pip package in the current Jupyter kernel
import sys
!{sys.executable} -m pip install beautifulsoup4

In [127]:
df = pd.read_csv("LEU0252881500A.csv") #csv from https://fred.stlouisfed.org/series/LEU0252881500A
#Employed full time: Median usual weekly nominal earnings (second quartile): Wage and salary workers: 16 years and over

In [15]:
# Add a third column that divides the second column's values by 40
df['median_hourly_pay'] = df.iloc[:, 1] / 40 #assuming 40 hour work week (full time assumption for US). It does not seem possible to get data on the median hours worked in the US for full time employees (same subset for weekly wage data)

# Display the updated DataFrame
print(df)

   observation_date  LEU0252881500A  median_hourly_pay
0        1979-01-01             241              6.025
1        1980-01-01             262              6.550
2        1981-01-01             284              7.100
3        1982-01-01             302              7.550
4        1983-01-01             313              7.825
5        1984-01-01             326              8.150
6        1985-01-01             344              8.600
7        1986-01-01             359              8.975
8        1987-01-01             374              9.350
9        1988-01-01             385              9.625
10       1989-01-01             399              9.975
11       1990-01-01             412             10.300
12       1991-01-01             426             10.650
13       1992-01-01             440             11.000
14       1993-01-01             459             11.475
15       1994-01-01             467             11.675
16       1995-01-01             479             11.975
17       1

In [53]:
# https://www.congress.gov/crs-product/RL34498 for tax rate
print(1159 * 52 - 15000) #accounting for standard deduction
ded = 1159 * 52 - 15000

45268


In [55]:
print(1192.5 + .12 * (ded-11925)) #"over $11,925 to $48,475 = $1,192.50 + 12% of the amount over $11,925"

5193.66


In [57]:
print(1159*52-5193.66)
afterfed = 1159*52-5193.66

55074.34


In [59]:
print((1159*52-afterfed)/(1159*52)) # federal tax rate %

0.08617608017521743


In [97]:
#now accounting for social security and medicare: https://www.irs.gov/pub/irs-pdf/p926.pdf
print(ded*.062+ ded*.0145)
ssmc = ded*.062+ ded*.0145
fin= afterfed -ssmc
print (fin)

3463.002
51611.337999999996


In [79]:
#scraping tax rates for states to find an average
import requests
from bs4 import BeautifulSoup

# URL for the state income tax rates
url = 'https://www.nerdwallet.com/article/taxes/state-income-tax-rates' #not the best source, but it is a source we can use.

# Step 1: Get the page content
response = requests.get(url)

# Step 2: Parse the page content
soup = BeautifulSoup(response.text, 'html.parser')

# Step 3: Find all tables on the page
tables = soup.find_all('table')

# Step 4: Select the second table (index 1 for the second table)
table = tables[1]

# Step 5: Extract table headers and rows
headers = [header.text.strip() for header in table.find_all('th')]
rows = table.find_all('tr')[1:]  # Skip the header row

# Step 6: Extract the data from each row
data = []
for row in rows:
    cols = row.find_all('td')
    cols = [col.text.strip() for col in cols]
    data.append(cols)

# Step 7: Create DataFrame and save it as CSV
df2 = pd.DataFrame(data, columns=headers)

In [77]:
df2 = df2.drop(index=51) #cleaning up extraneous information
df2

,State,Tax rates,Lowest and highest tax bracket (income)
0,Alabama,2%-5%.,"$500-$3,000."
1,Alaska,Does not have state income tax.,None
2,Arizona,2.5%.,Flat rate.
3,Arkansas,2%-3.9%.,"Income over $4,500 is subject to a 3.9% rate."
4,California,1%-13.3%.,"$10,756-$1,000,000."
5,Colorado,4.4%.,Flat rate.
6,Connecticut,2%-6.99%.,"$10,000-$500,000."
7,Delaware,2.2%-6.6%.,"$2,000-$60,000."
8,District of Columbia,4%-10.75%.,"$10,000-$1,000,000."
9,Florida,Does not have state income tax.,None


In [91]:
# Reconstruct the state tax data (used llm for this part once i had data scraped, probably a more novel approach is possible, but the llm one shot it so why bother)
data = {
    "State": [
        "Alabama", "Alaska", "Arizona", "Arkansas", "California", "Colorado",
        "Connecticut", "Delaware", "District of Columbia", "Florida", "Georgia",
        "Hawaii", "Idaho", "Illinois", "Indiana", "Iowa", "Kansas", "Kentucky",
        "Louisiana", "Maine", "Maryland", "Massachusetts", "Michigan", "Minnesota",
        "Mississippi", "Missouri", "Montana", "Nebraska", "Nevada", "New Hampshire",
        "New Jersey", "New Mexico", "New York", "North Carolina", "North Dakota",
        "Ohio", "Oklahoma", "Oregon", "Pennsylvania", "Rhode Island",
        "South Carolina", "South Dakota", "Tennessee", "Texas", "Utah", "Vermont",
        "Virginia", "Washington", "West Virginia", "Wisconsin", "Wyoming"
    ],
    "Tax rates": [
        "2%-5%", "None", "2.5%", "2%-3.9%", "1%-13.3%", "4.4%", "2%-6.99%", "2.2%-6.6%",
        "4%-10.75%", "None", "5.39%", "1.4%-11%", "5.695%", "4.95%", "3%", "3.8%",
        "5.2%-5.58%", "4%", "3%", "5.8%-7.15%", "2%-5.75%", "5%-9%", "4.25%",
        "5.35%-9.85%", "4.4%", "2%-4.7%", "4.7%-5.9%", "2.46%-5.2%", "None", "None",
        "1.4%-10.75%", "1.5%-5.9%", "4%-10.9%", "4.25%", "1.95%-2.5%", "2.75%-3.5%",
        "0.25%-4.75%", "4.75%-9.9%", "3.07%", "3.75%-5.99%", "0%-6.2%", "None", "None",
        "None", "4.55%", "3.35%-8.75%", "2%-5.75%", "None", "2.22%-4.82%", "3.5%-7.65%",
        "None"
    ]
}

# Create a DataFrame
tax_data = pd.DataFrame(data)

# Define the median income for calculation
median_income = 60268

# simplified approach for a median tax rate per state, not accounting for the exact marginal tax rates and progression that states may have.
def calculate_tax(rate_str, income):
    if rate_str == "None":
        return 0.0
    if "-" in rate_str:  # For progressive rates with a range
        low_rate, high_rate = map(lambda x: float(x.strip('%')) / 100, rate_str.split('-'))
        return income * ((low_rate + high_rate) / 2)  # Average of rates for simplicity
    else:  # For flat tax rates
        flat_rate = float(rate_str.strip('%')) / 100
        return income * flat_rate

# Reapply the tax calculation to all states
tax_data["State Tax for $60,000"] = tax_data["Tax rates"].apply(lambda rate: calculate_tax(rate, median_income))

# Recalculate the median state tax and effective tax rate
median_state_tax = tax_data["State Tax for $60,000"].median()
median_effective_tax_rate = median_state_tax / median_income

median_state_tax, median_effective_tax_rate

(np.float64(2335.3849999999998), np.float64(0.03874999999999999))

In [105]:
afteralltax25 = fin - median_state_tax
print(afteralltax25)

49275.952999999994


In [109]:
alltaxrate = (median_income-afteralltax25)/median_income
print(alltaxrate) # this alligns with my expectations, just more rigorous now than blindly trusting a value

0.18238612530696233


In [125]:
#while 2017 tcja and 2001 cuts are meaningful, for the median income they are not that massive, and have some offsets which may impact it. A much more
#rigorous effort involving immense tax analysis would have to be done to show a year to year difference if that rate itself has shifted. I am comfortable
#not spending that effort analyzing every single year and every single state tax code
df['annual after tax pay'] = df.iloc[:, 1] *52*(1-alltaxrate)
df['median hourly after tax pay'] = df.iloc[:, 3] /(52*40)
#net spendable amount of money, and per hour, is known (not accounting for shifts in cost of living that may have occurred as this entire project is one approach at quantifying such a stat)

,observation_date,LEU0252881500A,median_hourly_pay,annual after tax pay,median hourly after tax pay
0,1979-01-01,241,6.025,10246.337078,4.926124
1,1980-01-01,262,6.550,11139.171429,5.355371
2,1981-01-01,284,7.100,12074.521701,5.805059
3,1982-01-01,302,7.550,12839.808288,6.172985
4,1983-01-01,313,7.825,13307.483425,6.397829
5,1984-01-01,326,8.150,13860.190404,6.663553
6,1985-01-01,344,8.600,14625.476991,7.031479
7,1986-01-01,359,8.975,15263.215813,7.338085
8,1987-01-01,374,9.350,15900.954635,7.644690
9,1988-01-01,385,9.625,16368.629771,7.869534


In [ ]:
#on to the cost of a big mac now, which in modern day can be difficult to quantify with incredibly dynamic pricing (apps, discounts, regions, etc)
